<a href="https://colab.research.google.com/github/Riiishaab/MLEE/blob/main/MLEE_Exp_5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from scipy.stats import mode

## Sigmoid

In [ ]:
def sigmoid(z):
  return 1/(1 + np.exp(-z))

## Logistic Regression for Training

In [ ]:
# Batch Gradient Descent
def train_logistic_regression_BatchGD(X, y, learning_rate, epochs):
    samples, features = X.shape
    w = np.zeros(features)

    for epoch in range(epochs):
        z = np.dot(X, w)
        y_pred = sigmoid(z)

        # Compute gradient and update weights
        gradient = (1 / samples) * np.dot(X.T, (y_pred - y))
        w -= learning_rate * gradient

    return w

# Stochastic Gradient Descent
def train_logistic_regression_SGD(X, y, learning_rate, epochs):
    samples, features = X.shape
    w = np.zeros(features)

    for epoch in range(epochs):
        # Shuffle the data each epoch
        shuffled_indices = np.random.permutation(samples)
        X_shuffled = X[shuffled_indices]
        y_shuffled = y[shuffled_indices]

        # Update weights for each sample
        for i in range(samples):
            xi = X_shuffled[i]
            yi = y_shuffled[i]

            z = np.dot(xi, w)
            y_pred = sigmoid(z)

            # Compute gradient and update weights
            gradient = xi * (y_pred - yi)
            w -= learning_rate * gradient

    return w

# Mini-Batch Gradient Descent
def train_logistic_regression_MBGD(X, y, learning_rate, epochs, batch_size=32):
    samples, features = X.shape
    w = np.zeros(features)

    for epoch in range(epochs):
        # Shuffle the data each epoch
        shuffled_indices = np.random.permutation(samples)
        X_shuffled = X[shuffled_indices]
        y_shuffled = y[shuffled_indices]

        # Process mini-batches
        for i in range(0, samples, batch_size):
            X_batch = X_shuffled[i:i + batch_size]
            y_batch = y_shuffled[i:i + batch_size]

            z = np.dot(X_batch, w)
            y_pred = sigmoid(z)

            # Compute gradient and update weights
            gradient = (1 / len(X_batch)) * np.dot(X_batch.T, (y_pred - y_batch))
            w -= learning_rate * gradient

    return w


## Logistic Regression for Testing and Prediction

In [ ]:
def predict_logistic_regression(X,w):
  z = np.dot(X,w)
  y_pred = sigmoid(z)
  return np.where(y_pred > 0.5,1,0)

# One v/s All Multi-Class Classification

In [ ]:
import numpy as np

def train_one_vs_all(X, y, learning_rate, epochs):
    classes = len(np.unique(y))
    classifiers_BGD = {}
    classifiers_SGD = {}
    classifiers_MBGD = {}

    for c in range(classes):
        print(f"Training Classification Model for Class {c}")
        y_binary = np.where(y == c, 1, 0)

        # Train models using each gradient descent approach
        w_BatchGD = train_logistic_regression_BatchGD(X, y_binary, learning_rate, epochs)
        w_StochasticGD = train_logistic_regression_SGD(X, y_binary, learning_rate, epochs)
        w_MBGD = train_logistic_regression_MBGD(X, y_binary, learning_rate, epochs)

        # Store weights for each class
        classifiers_BGD[c] = w_BatchGD
        classifiers_SGD[c] = w_StochasticGD
        classifiers_MBGD[c] = w_MBGD

    return classifiers_BGD, classifiers_SGD, classifiers_MBGD


In [ ]:
def predict_one_vs_all(X, classifiers):
    probabilities = {}

    # Compute class probabilities for each classifier
    for c, w in classifiers.items():
        probabilities[c] = sigmoid(np.dot(X, w))

    # Find the class with the highest probability for each sample
    predictions = np.array([
        max(probabilities, key=lambda k: probabilities[k][i])
        for i in range(X.shape[0])
    ])

    return predictions

# One v/s One Multi Class Classification

In [ ]:
def train_one_vs_one(X, y, learning_rate, epochs):
    classes = np.unique(y)
    classifiers_BGD = []
    classifiers_SGD = []
    classifiers_MBGD = []
    pairs_BGD = []
    pairs_SGD = []
    pairs_MBGD = []

    for i, c1 in enumerate(classes):
        for c2 in classes[i+1:]:
            idx = np.where((y == c1) | (y == c2))[0]
            X_pair = X[idx]
            y_pair = np.where(y[idx] == c1, 1, 0)

            print(f"Training {c1} vs {c2} classifier")

            # Train different variants
            w_BGD = train_logistic_regression_BatchGD(X_pair, y_pair, learning_rate, epochs)
            w_SGD = train_logistic_regression_SGD(X_pair, y_pair, learning_rate, epochs)  # Corrected here
            w_MBGD = train_logistic_regression_MBGD(X_pair, y_pair, learning_rate, epochs)

            # Store results for each variant
            classifiers_BGD.append(w_BGD)
            pairs_BGD.append((c1, c2))

            classifiers_SGD.append(w_SGD)
            pairs_SGD.append((c1, c2))

            classifiers_MBGD.append(w_MBGD)
            pairs_MBGD.append((c1, c2))

    return classifiers_BGD, classifiers_SGD, classifiers_MBGD, pairs_BGD, pairs_SGD, pairs_MBGD


In [ ]:
def predict_one_vs_one(X, classifiers, pairs):
    predictions = []

    # Predict class for each classifier
    for (c1, c2), weights in zip(pairs, classifiers):
        pred = predict_logistic_regression(X, weights)
        pred_classes = np.where(pred == 1, c1, c2)
        predictions.append(pred_classes)

    # Stack predictions and take the mode
    predictions = np.array(predictions).T
    final_predictions = mode(predictions, axis=1, keepdims=False).mode.flatten()

    return final_predictions


# Evalution Metrices

In [ ]:
from sklearn.metrics import confusion_matrix,classification_report,cohen_kappa_score

# Function to evaluate metrics
def evaluate_metrics(y_true, y_pred, method_name=""):
    print(f"\n--- Metrics for {method_name} ---")
    # Confusion Matrix
    cm = confusion_matrix(y_true, y_pred)
    print(f"Confusion Matrix:\n{cm}")
    # Precision, Recall, F1-Score
    report = classification_report(y_true, y_pred, digits=4)
    print(f"Classification Report:\n{report}")
    # Kappa Score
    kappa = cohen_kappa_score(y_true, y_pred)
    print(f"Kappa Score: {kappa:.4f}")
    print("-" * 40)

# Add bias column
def add_bias(X):
    return np.hstack((X, np.ones((X.shape[0], 1))))

import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.datasets import make_classification
from sklearn.metrics import confusion_matrix, classification_report, cohen_kappa_score

# Function to evaluate metrics
def evaluate_metrics(y_true, y_pred, method_name=""):
    print(f"\n--- Metrics for {method_name} ---")
    # Confusion Matrix
    cm = confusion_matrix(y_true, y_pred)
    print(f"Confusion Matrix:\n{cm}")
    # Precision, Recall, F1-Score
    report = classification_report(y_true, y_pred, digits=4)
    print(f"Classification Report:\n{report}")
    # Kappa Score
    kappa = cohen_kappa_score(y_true, y_pred)
    print(f"Kappa Score: {kappa:.4f}")
    print("-" * 40)

# Add bias column
def add_bias(X):
    return np.hstack((X, np.ones((X.shape[0], 1))))

# Example Dataset
X, y = make_classification(n_samples=300,
                           n_features=5,
                           n_classes=3,
                           n_informative=3,
                           random_state=42)

X = add_bias(X)  # Add bias column
X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                    test_size=0.2,
                                                    random_state=42)

import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.datasets import make_classification
from sklearn.metrics import confusion_matrix, classification_report, cohen_kappa_score

# Function to evaluate metrics
def evaluate_metrics(y_true, y_pred, method_name=""):
    print(f"\n--- Metrics for {method_name} ---")
    # Confusion Matrix
    cm = confusion_matrix(y_true, y_pred)
    print(f"Confusion Matrix:\n{cm}")
    # Precision, Recall, F1-Score
    report = classification_report(y_true, y_pred, digits=4)
    print(f"Classification Report:\n{report}")
    # Kappa Score
    kappa = cohen_kappa_score(y_true, y_pred)
    print(f"Kappa Score: {kappa:.4f}")
    print("-" * 40)

# Add bias column
def add_bias(X):
    return np.hstack((X, np.ones((X.shape[0], 1))))

# Example Dataset
X, y = make_classification(n_samples=300,
                           n_features=5,
                           n_classes=3,
                           n_informative=3,
                           random_state=42)

X = add_bias(X)  # Add bias column
X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                    test_size=0.2,
                                                    random_state=42)

# Example Dataset
X, y = make_classification(n_samples=300, n_features=5, n_classes=3,
n_informative=3, random_state=42)
X = add_bias(X) # Add bias column
X_train, X_test, y_train, y_test = train_test_split(X, y,
test_size=0.2, random_state=42)


# One v/s All Implementation

In [ ]:
from sklearn.metrics import accuracy_score

# Train one-vs-all classifiers
classifiers_BGD, classifiers_SGD, classifiers_MBGD = train_one_vs_all(
    X_train, y_train,
    learning_rate=0.1, epochs=2000
)

# Predict with Batch Gradient Descent (BGD)
y_pred_BGD = predict_one_vs_all(X_test, classifiers_BGD)
print(f"One-vs-All Accuracy of BGD: {accuracy_score(y_test, y_pred_BGD):.4f}")

# Predict with Stochastic Gradient Descent (SGD)
y_pred_SGD = predict_one_vs_all(X_test, classifiers_SGD)
print(f"One-vs-All Accuracy of SGD: {accuracy_score(y_test, y_pred_SGD):.4f}")

# Predict with Mini-Batch Gradient Descent (MBGD)
y_pred_MBGD = predict_one_vs_all(X_test, classifiers_MBGD)
print(f"One-vs-All Accuracy of MBGD: {accuracy_score(y_test, y_pred_MBGD):.4f}")

# Evaluate metrics for each variant
evaluate_metrics(y_test, y_pred_BGD, method_name="One-vs-All-BGD")
evaluate_metrics(y_test, y_pred_SGD, method_name="One-vs-All-SGD")
evaluate_metrics(y_test, y_pred_MBGD, method_name="One-vs-All-MBGD")


Training Classification Model for Class 0
Training Classification Model for Class 1
Training Classification Model for Class 2
One-vs-All Accuracy of BGD: 0.6667
One-vs-All Accuracy of SGD: 0.6167
One-vs-All Accuracy of MBGD: 0.6667

--- Metrics for One-vs-All-BGD ---
Confusion Matrix:
[[16  3  1]
 [ 6 10  7]
 [ 3  0 14]]
Classification Report:
              precision    recall  f1-score   support

           0     0.6400    0.8000    0.7111        20
           1     0.7692    0.4348    0.5556        23
           2     0.6364    0.8235    0.7179        17

    accuracy                         0.6667        60
   macro avg     0.6819    0.6861    0.6615        60
weighted avg     0.6885    0.6667    0.6534        60

Kappa Score: 0.5056
----------------------------------------

--- Metrics for One-vs-All-SGD ---
Confusion Matrix:
[[12  7  1]
 [ 6 12  5]
 [ 3  1 13]]
Classification Report:
              precision    recall  f1-score   support

           0     0.5714    0.6000    0.5854

# One v/s One Implementation

In [ ]:
from sklearn.metrics import accuracy_score

# Train classifiers for one-vs-one
classifiers_BGD, classifiers_SGD, classifiers_MBGD, pairs_BGD, pairs_SGD, pairs_MBGD = train_one_vs_one(
    X_train, y_train,
    learning_rate=0.1, epochs=2000
)

# Predictions using one-vs-one classifiers
y_pred_BGD = predict_one_vs_one(X_test, classifiers_BGD, pairs_BGD)
y_pred_SGD = predict_one_vs_one(X_test, classifiers_SGD, pairs_SGD)
y_pred_MBGD = predict_one_vs_one(X_test, classifiers_MBGD, pairs_MBGD)

# Print accuracies for each method
print(f"One-vs-One Accuracy BGD: {accuracy_score(y_test, y_pred_BGD):.4f}")
print(f"One-vs-One Accuracy SGD: {accuracy_score(y_test, y_pred_SGD):.4f}")
print(f"One-vs-One Accuracy MBGD: {accuracy_score(y_test, y_pred_MBGD):.4f}")

# Evaluate metrics for each method
evaluate_metrics(y_test, y_pred_BGD, method_name="One-vs-One-BGD")
evaluate_metrics(y_test, y_pred_SGD, method_name="One-vs-One-SGD")
evaluate_metrics(y_test, y_pred_MBGD, method_name="One-vs-One-MBGD")


Training 0 vs 1 classifier
Training 0 vs 2 classifier
Training 1 vs 2 classifier
One-vs-One Accuracy BGD: 0.6333
One-vs-One Accuracy SGD: 0.6667
One-vs-One Accuracy MBGD: 0.6500

--- Metrics for One-vs-One-BGD ---
Confusion Matrix:
[[13  6  1]
 [ 4 13  6]
 [ 3  2 12]]
Classification Report:
              precision    recall  f1-score   support

           0     0.6500    0.6500    0.6500        20
           1     0.6190    0.5652    0.5909        23
           2     0.6316    0.7059    0.6667        17

    accuracy                         0.6333        60
   macro avg     0.6335    0.6404    0.6359        60
weighted avg     0.6329    0.6333    0.6321        60

Kappa Score: 0.4486
----------------------------------------

--- Metrics for One-vs-One-SGD ---
Confusion Matrix:
[[15  4  1]
 [ 6 12  5]
 [ 4  0 13]]
Classification Report:
              precision    recall  f1-score   support

           0     0.6000    0.7500    0.6667        20
           1     0.7500    0.5217    0.6154